# Descriptors

Classes that implement the descriptor protocol and allow specialised behaviour when accessing atttribues, such validation or logging. See https://docs.python.org/3/howto/descriptor.html.

The descriptor interface includes the following methods:
* `__get__()`
* `__set__()`
* `__delete__()`
* `__set_name__()`

These are called when accessing an attribute of an object with the `.` (dot) operator.

If class only implements `__get__` then it is a _non-data descriptor_, else it is called a _data-descriptor_. So let's start with a non-data descriptor which only implement the `__get__` method.

In [24]:
class MyLoggingAgeNonDataDescriptor:
    def __get__(self, obj, type=None):
        """by defining get makes this a descriptor type"""
        print("MyLoggingAgeNonDataDescriptor: someone is accessing this constant via __get__")
        return 16

class MyClass:
    """descriptors live in class and not instance"""
    min_age = MyLoggingAgeNonDataDescriptor()

my_instance = MyClass()
print(my_instance.min_age)  # overrides normal attribute look up and we can log the fact


MyLoggingAgeNonDataDescriptor: someone is accessing this constant via __get__
16


`obj` in the `__get__` function refers to the object to which this descriptor object is bound. We can use that to perform more dynamic behaviour when a user access the attribute.

In [27]:
class MyLoggingAgeNonDataDescriptor:
    def __get__(self, obj, type=None):
        print("MyLoggingAgeNonDataDescriptor: __get__")
        return 16 if obj.is_uk else 18     # obj is a MyClass instance so we can return different constants

class MyClass():
    min_age = MyLoggingAgeNonDataDescriptor()
    def __init__(self, is_uk):
        self.is_uk = is_uk

my_instance = MyClass(True)
print(my_instance.min_age)
my_instance = MyClass(False)
print(my_instance.min_age)

MyLoggingAgeNonDataDescriptor: __get__
16
MyLoggingAgeNonDataDescriptor: __get__
18


Whenever we access the `min_age` attribute of any instance of the class then `__get__` will be called

In [30]:
print("type(my_instance.min_age) =", str(type(my_instance.min_age))[1:-1])

MyLoggingAgeNonDataDescriptor: __get__
type(my_instance.min_age) = class 'int'


A data descriptor allows you both to customize both the setting and getting an atttibute. Here the public attribute `my_age` is a data descriptor. The value is actually stored in an object instance, `_my_age`.

In [39]:
class MyLoggingAgeDataDescriptor:
    def __get__(self, obj, type=None):
        print("MyAgeDataDescriptor: __get__ getting age")
        return obj._my_age

    def __set__(self, obj, value):
        print("MyAgeDataDescriptor: __set__ setting new age")
        obj._my_age = value

class MyClass:
    my_age = MyLoggingAgeDataDescriptor()
    def __init__(self, age):
        self.my_age = age     # will call __set__ on my_age descriptor

print("Create instances")
my_instance_young = MyClass(14)
my_instance_old = MyClass(89)
print("Changing age of instances")
my_instance_young.my_age = 15
my_instance_old.my_age = 90
print("Print descriptor")
print(my_instance_young.my_age)
print(my_instance_old.my_age)

Create instances
MyAgeDataDescriptor: __set__ setting new age
MyAgeDataDescriptor: __set__ setting new age
Changing age of instances
MyAgeDataDescriptor: __set__ setting new age
MyAgeDataDescriptor: __set__ setting new age
Print descriptor
MyAgeDataDescriptor: __get__ getting age
15
MyAgeDataDescriptor: __get__ getting age
90


See https://docs.python.org/3/howto/descriptor.html for how to use `__setname__` to write a single descriptor that can support multiple attributes i.e. you don't have to write a class for every attribute you want to log.

Using a read-only descriptor you can implement read-only.

In [40]:
class MyReadOnlyDescriptor:
    def __get__(self, obj, type=None):
        print("MyReadOnlyDescriptor: __get__")
        return "Hello!"

    def __set__(self, obj, value):
        print("MyReadOnlyDescriptor: __set__")
        raise AttributeError("This value is read only!")

class MyClass:
    my_readonly_descriptor = MyReadOnlyDescriptor()
    def __init__(self):
        pass

my_instance = MyClass()
print(my_instance.my_readonly_descriptor)
try:
    my_instance.my_readonly_descriptor = "Try and change this value"    # recommended way to implement
except AttributeError as e:                                             # read-only attribute
    print("Exception:", e)

MyReadOnlyDescriptor: __get__
Hello!
MyReadOnlyDescriptor: __set__
Exception: This value is read only!


### What is `self`?

In [41]:
class MySelf:
    def __init__(self):
        print("__init__: initialisation")
        print("__init__: type(self) =", str(type(self))[1:-1])
        print("__init__: id(self) =", id(self))
        print(self)                                     # what is self?, self is a MySelf object
        self.some_attribute = 'some instance attribute'

my_instance = MySelf()

__init__: initialisation
__init__: type(self) = class '__main__.MySelf'
__init__: id(self) = 2524343137232


You don't need to call it `self` - it's just the accepted name to use for first parameter in a bound method.

In [44]:
class MySelf:
    def __init__(self):
        self.some_attribute = "Some attribute"

    def do_something(funky_chicken):
        print("do_something: type(funky_chicken) =", str(type(funky_chicken))[1:-1])
        print("do_something: id(funky_chicken) =", id(funky_chicken))
        print("do_something: funky_chicken.some_attribute =", funky_chicken.some_attribute)

my_instance = MySelf()
my_instance.do_something()

do_something: type(funky_chicken) = class '__main__.MySelf'
do_something: id(funky_chicken) = 2524344365456
do_something: funky_chicken.some_attribute = Some attribute


### What is the `__dict__` attribute and the look up chain?

The look up chain is the order in which Python discovers an attribute. For example is I call `print(my_instance.my_class_variable)` then the interpreter will look to see if `my_class_variable` is a data descriptor that exposes the `__get__` part of the data descriptor interface. If that fails then it will try `my_instance.__dict__['my_class_variable']` and failing that, the lookup chain will look for a non-data descriptors with a `__get__` method, followed by type `(my_instance).__dict__['my_class_variable']` (or `MyClass.__dict__['my_class_variable']`) and failing all that will start progressing up parent classes.

In [45]:
class MyClass:
    my_class_variable = 1                   # not a descriptor, will appear in MyClass.__dict__

    def __init__(self):
        self.my_instance_variable = 2       # will appear in my_instance.__dict__

my_instance = MyClass()
print(my_instance.__dict__)
print(MyClass.__dict__)
print(type(my_instance).__dict__)

{'my_instance_variable': 2}
{'__module__': '__main__', 'my_class_variable': 1, '__init__': <function MyClass.__init__ at 0x0000024BBEB19000>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'__module__': '__main__', 'my_class_variable': 1, '__init__': <function MyClass.__init__ at 0x0000024BBEB19000>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


### Monkey Patching

Dynamically adding/removing attributes at run-time to an object instance is known as a monkey patch.

In [47]:
import types

def my_dynamic_method(self, x):
    print(f"self.my_instance_variable + {x} =", self.my_instance_variable + x)

# monkey patch = modification/addition of attributes at run-time
my_instance.my_method = types.MethodType(my_dynamic_method, my_instance)
my_instance.my_method(5)            # method only available to this instance
del my_instance.my_method

self.my_instance_variable + 5 = 7


### Descriptors and instances

Use descriptor object to call instance method...

In [52]:
class MyFunctionDescriptor:
    def __get__(self, obj, type=None):
        print("MyFunctionDescriptor: __get__")
        return obj.do_something

    def __set__(self, obj, value):
        raise AttributeError("This value is read only!")


class MyDynamicFunctionDescriptor:
    def __get__(self, obj, type=None):
        return types.MethodType(my_dynamic_method, obj)     # similar to how instance methods are called

    def __set__(self, obj, value):
        raise AttributeError("This value is read only!")

class MySelf:
    my_do_something = MyFunctionDescriptor()
    my_dynamic_something = MyDynamicFunctionDescriptor()

    def __init__(self):
        self.my_instance_variable = 2

    def do_something(self):
        print("do_something: ok")

my_instance = MySelf()
my_instance.my_do_something()
my_instance.my_dynamic_something(19)

MyFunctionDescriptor: __get__
do_something: ok
self.my_instance_variable + 19 = 21


To get descriptors to work with instances, not just classes...

In [2]:
class MyInstanceDescriptor:
    def __set_name__(self, owner, name):                # available since Python 3.6
        print("MyInstanceDescriptor: __set_name__, owner =", str(owner())[1:-1], " name =", name)
        self.name = name

    def __get__(self, obj, type=None):
        print("MyInstanceDescriptor: __get__")
        return obj.__dict__.get(self.name) or None      # should be able to use getattr here

    def __set__(self, obj, value):
        print("MyInstanceDescriptor: __set__")
        obj.__dict__[self.name] = value                 # store value in object instance dictionary

class MyClass:
    print("MyClass: create instance descriptor")
    age = MyInstanceDescriptor()

    def __init__(self):
        pass

my_instance1 = MyClass()
my_instance2 = MyClass()

print("my_instance1.__dict__ =", my_instance1.__dict__)
print("my_instance2.__dict__ =", my_instance2.__dict__)
# MyClass.my_instance_descriptor = 3                     # this overwrites the descriptor with the 3 object..
my_instance1.age = 9                                     # ...rather than call __set__ with obj = class object
my_instance2.age = 14
print("my_instance1.__dict__ =", my_instance1.__dict__)
print("my_instance2.__dict__ =", my_instance2.__dict__)
print(my_instance1.age)
print(my_instance2.age)

MyClass: create instance descriptor
MyInstanceDescriptor: __set_name__, owner = __main__.MyClass object at 0x00000161FA4C7850  name = age
my_instance1.__dict__ = {}
my_instance2.__dict__ = {}
MyInstanceDescriptor: __set__
MyInstanceDescriptor: __set__
my_instance1.__dict__ = {'age': 9}
my_instance2.__dict__ = {'age': 14}
MyInstanceDescriptor: __get__
9
MyInstanceDescriptor: __get__
14
